# Bitquery


Bitquery provides GraphQL APIs for more than 30 blockchains.

The following blog post provides an introduction of Bitquery's integration of Cronos chain: https://community.bitquery.io/t/cronos-cro-apis-transfers-trades-tokens-dex/1289

## Explorer

The Cronos explorer can be used to perform simple queries: visit https://explorer.bitquery.io/cronos/

## Online IDE

For GraphQL queries, the first step is to use the IDE to draft the query:

- https://graphql.bitquery.io/ide
- Use ethereum query, with "cronos" as the network

Example of query to calculate monthly active users on Cronos chain:

```graphql
query MyQuery {
  ethereum(network: cronos) {
    transactions(date: { since: "2022-08-05" }) {
      date {
        date(format: "%Y-%m")
      }
      transactions: count
      senders: count(uniq: receivers)
      receivers: count(uniq: senders)
    }
  }
}
```

## API

With the Bitquery API, it is possible to send queries programmatically in GraphQL format and receive results in JSON. An API key is required.

Below are a few examples.

### Monthly active users on Cronos chain

In [23]:
import os
import requests
import pprint
import pandas
from IPython.display import display
from dotenv import load_dotenv  # poetry add python-dotenv
load_dotenv()

bitquery_api_key = os.environ.get('BITQUERY_API_KEY')

# A simple function to use requests.post to make the API call.
def run_query(query):
    headers = {'X-API-KEY': bitquery_api_key}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}  {}'.format(request.status_code,
                        query))

# The GraphQL query
query = """
query MyQuery {
  ethereum(network: cronos) {
    transactions(date: { since: "2021-11-01" }) {
      date {
        date(format: "%Y-%m")
      }
      transactions: count
      senders: count(uniq: senders)
      receivers: count(uniq: receivers)
    }
  }
}
"""

result = run_query(query)  # Execute the query

# Summarize in a dataframe
columns = ['month', 'senders', 'receivers', 'transactions']
df = pandas.DataFrame(data=[], columns=columns)
for datapoint in result['data']['ethereum']['transactions']:
  new_df = pandas.DataFrame(data=[[
    datapoint['date']['date'],
    datapoint['senders'],
    datapoint['receivers'],
    datapoint['transactions']
  ]], columns=columns)
  df = pandas.concat(objs=[df, new_df], ignore_index=True)

styled_df = df.style.format({
    "senders": "{:,}",
    "receivers": "{:,}",
    "transactions": "{:,}",
})

display(styled_df)


,month,senders,receivers,transactions
0,2021-11,"118,656","81,931","3,928,882"
1,2021-12,"197,526","112,115","5,160,935"
2,2022-01,"173,684","93,304","4,059,040"
3,2022-02,"177,332","98,916","5,298,797"
4,2022-03,"222,244","138,001","7,539,167"
5,2022-04,"289,446","221,468","10,671,468"
6,2022-05,"225,419","143,986","7,609,421"
7,2022-06,"164,536","105,004","4,561,982"
8,2022-07,"143,163","87,446","3,381,992"
9,2022-08,"137,121","81,209","3,387,389"


### Monthly active users sending transactions to VVS

In [25]:

# The GraphQL query
query = """
query MyQuery {
  ethereum(network: cronos) {
    transactions(
      date: { since: "2021-11-01" }
      txTo: {is: "0x145863Eb42Cf62847A6Ca784e6416C1682b1b2Ae"}
      ) {
      date {
        date(format: "%Y-%m")
      }
      transactions: count
      senders: count(uniq: senders)
      receivers: count(uniq: receivers)
    }
  }
}
"""

result = run_query(query)  # Execute the query

# Summarize in a dataframe
columns = ['month', 'senders', 'receivers', 'transactions']
df = pandas.DataFrame(data=[], columns=columns)
for datapoint in result['data']['ethereum']['transactions']:
  new_df = pandas.DataFrame(data=[[
    datapoint['date']['date'],
    datapoint['senders'],
    datapoint['receivers'],
    datapoint['transactions']
  ]], columns=columns)
  df = pandas.concat(objs=[df, new_df], ignore_index=True)

styled_df = df.style.format({
    "senders": "{:,}",
    "receivers": "{:,}",
    "transactions": "{:,}",
})

display(styled_df)


,month,senders,receivers,transactions
0,2021-11,"68,547",1,"894,419"
1,2021-12,"99,490",1,"1,098,306"
2,2022-01,"68,935",1,"549,172"
3,2022-02,"57,469",1,"448,470"
4,2022-03,"53,010",1,"419,885"
5,2022-04,"48,971",1,"386,217"
6,2022-05,"38,009",1,"342,261"
7,2022-06,"28,855",1,"300,754"
8,2022-07,"25,050",1,"246,897"
9,2022-08,"26,299",1,"266,665"
